# AIMO3 First Attempt (Composable + Kaggle-Automated)

This notebook is a high-quality first baseline focused on:
- `gpt-oss-120b` (or any OpenAI-compatible endpoint)
- multi-attempt solving with weighted aggregation
- tool-integrated reasoning via sandboxed Python code execution
- automated Kaggle submission and status polling via API

The code is shared with the package in `src/aimo3`, so improvements in one place are reusable everywhere.

In [ ]:
# If running in a clean Kaggle/Colab runtime, uncomment:
# !pip install -q kaggle pandas requests python-dotenv

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd

ROOT = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from aimo3.client import OpenAICompatChatClient
from aimo3.kaggle_api import KaggleAutomation
from aimo3.pipeline import run_inference, save_debug, save_submission
from aimo3.solver import AIMO3Solver, SolverConfig

print("Project root:", ROOT)

In [ ]:
# Runtime config
AIMO_MODEL = os.getenv("AIMO_MODEL", "openai/gpt-oss-120b")
AIMO_BASE_URL = os.getenv("AIMO_BASE_URL", "http://127.0.0.1:8000/v1")
AIMO_API_KEY = os.getenv("AIMO_API_KEY")

solver_cfg = SolverConfig(
    attempts=8,
    temperatures=(0.15, 0.25, 0.35, 0.45),
    max_tokens=1024,
    min_consensus=3,
    early_stop_attempt=4,
    max_code_blocks_per_attempt=2,
    default_answer=0,
)

client = OpenAICompatChatClient(
    base_url=AIMO_BASE_URL,
    model=AIMO_MODEL,
    api_key=AIMO_API_KEY,
    timeout_sec=180,
    extra_body={"top_p": 0.95},
)

solver = AIMO3Solver(client, config=solver_cfg)
print("Solver ready with model:", AIMO_MODEL)

In [ ]:
# Smoke test on a toy problem before full runs
toy_problem = "Compute 3^20 modulo 100000."
result = solver.solve(problem_text=toy_problem, problem_id="toy-1")
print(result.debug_summary)

In [ ]:
# Full batch inference (expects columns: id, problem)
INPUT_CSV = ROOT / "examples" / "sample_problems.csv"  # replace with Kaggle test.csv path
SUBMISSION_CSV = ROOT / "artifacts" / "submission.csv"
DEBUG_JSON = ROOT / "artifacts" / "debug_traces.json"

problems = pd.read_csv(INPUT_CSV)
submission_df, debug_rows = run_inference(solver, problems, id_col="id", problem_col="problem", verbose=True)

save_submission(submission_df, SUBMISSION_CSV)
save_debug(debug_rows, DEBUG_JSON)

print("Submission saved:", SUBMISSION_CSV)
print("Debug traces saved:", DEBUG_JSON)
submission_df.head()

In [ ]:
# Kaggle API automation: submit and poll status
# Requires KAGGLE_USERNAME + KAGGLE_KEY in environment (or KAGGLE_API_TOKEN=username:key)
COMPETITION = "ai-mathematical-olympiad-progress-prize-3"
MESSAGE = "AIMO3 first composable baseline (gpt-oss-120b + sandbox)"

api = KaggleAutomation(COMPETITION)
print(api.submit(SUBMISSION_CSV, message=MESSAGE))

# Optional polling until scored
final_state = api.wait_for_latest(poll_seconds=20, timeout_minutes=60)
print(final_state)